In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [ ]:
front_2Dcoord = np.load('front_image.npy', allow_pickle=True)
back_2Dcoord = np.load('back_image.npy', allow_pickle=True)
front_img = cv2.imread('front.png')

In [ ]:
#visualization
print(front_2Dcoord.shape, back_2Dcoord.shape)
from copy import deepcopy
vis_img = deepcopy(front_img)
for i in range(front_2Dcoord.shape[0]):
    cv2.circle(vis_img, (int(front_2Dcoord[i,0]), int(front_2Dcoord[i,1])), 2, (255,0,0), 2)
plt.imshow(vis_img[:,:,::-1])
plt.show()

In [ ]:
# ------------TODO----------------
# Compute corresponding 3D coordinate
t = np.array([[0, 0, 0], [50, 0, 0], [50, 50, 0], [0, 50, 0]])
u = t + np.array([160, 240, 0])
v = t + np.array([400, 400, 0])
front_3Dcoord = np.vstack((t, u, v)).astype(np.float64)
back_3Dcoord = front_3Dcoord + np.array([0, 0, 150])
# ------------TODO----------------

all_2Dcoord = np.concatenate((front_2Dcoord, back_2Dcoord), axis=0)
all_3Dcoord = np.concatenate((front_3Dcoord, back_3Dcoord), axis=0)
print(all_2Dcoord.shape, all_3Dcoord.shape)

In [ ]:
# ------------TODO----------------
# Construct the Equation Pm=0 and solve m
def f(p, u, v):
    p = np.hstack((p, 1))
    z = np.zeros((4,))
    l0 = np.hstack((p.T, z, -u * p.T))
    l1 = np.hstack((z, p.T, -v * p.T))
    return np.vstack((l0, l1))
g = lambda x: f(x[:3], x[3], x[4])
n = all_2Dcoord.shape[0]
P = np.hstack((all_3Dcoord, all_2Dcoord))
P = np.apply_along_axis(g, 1, P).reshape(2 * n, -1)
U, S, Vt = np.linalg.svd(P)
m = Vt[-1]
# ------------TODO----------------

In [ ]:
# ------------TODO----------------
# Solve K, and [R T] from m
M = m.reshape(3, 4)
A = M[:, :3]
b = M[:, 3]
a1, a2, a3 = map(lambda x: x.reshape(-1), np.vsplit(A, 3))

rho_sq = 1 / np.dot(a3, a3) # How to decide the sign?
cx = rho_sq * np.dot(a1, a3)
cy = rho_sq * np.dot(a2, a3)
t1 = np.cross(a2, a3)
t2 = np.cross(a3, a1)
t1_norm = np.linalg.norm(t1)
t2_norm = np.linalg.norm(t2)
cos_theta = np.dot(t1, t2) / (t1_norm * t2_norm)
sin_theta = np.sqrt(1 - cos_theta ** 2)
cot_theta = cos_theta / sin_theta

alpha = rho_sq * t2_norm * sin_theta
beta = rho_sq * t1_norm * sin_theta

r1 = t1 / t1_norm
r3 = a3 / np.linalg.norm(a3)
r2 = np.cross(r3, r1)

K = np.array([
    [alpha, -alpha * cot_theta, cx],
    [0, beta / sin_theta, cy],
    [0, 0, 1]
])
R = np.vstack((r1, r2, r3))
T = np.sqrt(rho_sq) * np.linalg.inv(K) @ b
# ------------TODO----------------

In [ ]:
# save results
save_dict = {'3Dcoord': all_3Dcoord, 'P':P, 'm':m, 'K':K, 'R':R, 'T':T}
np.save('../results/calibr', save_dict)

# The reference answer of T should be [383001.8386981  -67357.81794783  67071.11338303]
# The results are very strange because the data is fake.